In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,losses,optimizers,datasets

# 获取所有GPU 设备列表
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 设置GPU 显存占用为按需分配
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # 异常处理
        print(e)

1 Physical GPUs, 1 Logical GPUs


**全连接网络的问题**

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model=keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(256,activation='relu'),
    layers.Dense(256,activation='relu'),
    layers.Dense(256,activation='relu'),
    layers.Dense(10),
  ]
)

model.build(input_shape=(4, 784))
model.summary()

11501568/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 335,114
Trainable params: 335,114
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer='adam',
      loss=losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'],)
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 8s 3ms/step - loss: 0.2027 - accuracy: 0.9384
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0915 - accuracy: 0.9719
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0660 - accuracy: 0.9793
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0526 - accuracy: 0.9834
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0407 - accuracy: 0.9870
313/313 - 1s - loss: 0.0749 - accuracy: 0.9791


[0.07493585348129272, 0.9790999889373779]

**卷积层实现**

In [4]:
#卷积层实现
x = tf.random.normal([2,5,5,4]) # 模拟输入，3通道，高宽为5
# 需要根据[k,k,cin,cout]格式创建，4个卷积核
w = tf.random.normal([3,3,4,4]) 
# 步长为1, padding为0,
out = tf.nn.conv2d(x,w,strides=1,padding=[[0,0],[0,0],[0,0],[0,0]])
print(out.shape)

(2, 3, 3, 4)


In [5]:
x = tf.random.normal([2,5,5,3]) # 模拟输入，3通道，高宽为5
# 需要根据[k,k,cin,cout]格式创建，4个卷积核
w = tf.random.normal([3,3,3,4])
# 步长为1, padding为1,
out = tf.nn.conv2d(x,w,strides=1,padding=[[0,0],[1,1],[1,1],[0,0]])
print(out.shape)

(2, 5, 5, 4)


In [6]:
x = tf.random.normal([2,5,5,3]) # 模拟输入，3通道，高宽为5
w = tf.random.normal([3,3,3,4]) # 4个3x3大小的卷积核
# 步长为,padding设置为输出、输入同大小
# 需要注意的是, padding=same只有在strides=1时才是同大小
out = tf.nn.conv2d(x,w,strides=1,padding='SAME')
print(out.shape)

(2, 5, 5, 4)


In [7]:
x = tf.random.normal([2,5,5,3])
w = tf.random.normal([3,3,3,4])
# 高宽按3倍减少
out = tf.nn.conv2d(x,w,strides=3,padding='SAME')
print(out.shape)

(2, 2, 2, 4)


In [8]:
# 根据[cout]格式创建偏置向量
b = tf.zeros([4])
# 在卷积输出上叠加偏置向量，它会自动broadcasting为[b,h',w',cout]
out = out + b
print(out.shape)

(2, 2, 2, 4)


**卷积层类实现**

In [9]:
# 创建卷积层类
layer = layers.Conv2D(4,kernel_size=(3,4),strides=(2,1),padding='SAME')
out = layer(x) # 前向计算
print(out.shape)

(2, 3, 5, 4)


In [10]:
layer.kernel, layer.bias
# 返回所有待优化张量列表
layer.trainable_variables

[<tf.Variable 'conv2d/kernel:0' shape=(3, 4, 3, 4) dtype=float32, numpy=
 array([[[[-0.26209965,  0.16388807,  0.01635584,  0.08101431],
          [-0.16938764, -0.06582133,  0.01951101,  0.2507122 ],
          [ 0.03945342, -0.11431898, -0.08431551, -0.21058685]],
 
         [[-0.20895237, -0.06279209,  0.08866996, -0.06119704],
          [ 0.22435999,  0.2084823 ,  0.00640693,  0.18406343],
          [-0.22250575, -0.03882182, -0.16251536,  0.1787397 ]],
 
         [[ 0.1623609 ,  0.11752799,  0.0971626 , -0.18415251],
          [-0.07323842, -0.00301072, -0.12026706, -0.08235319],
          [-0.24331996, -0.20121668, -0.19132122, -0.2440379 ]],
 
         [[ 0.18039483, -0.1421765 ,  0.2528639 ,  0.0490694 ],
          [-0.1730769 ,  0.15489751,  0.1910406 ,  0.14108345],
          [-0.17354703, -0.0317104 ,  0.12377295, -0.17848462]]],
 
 
        [[[-0.02776594, -0.19117033, -0.15911525, -0.08171031],
          [ 0.12114272,  0.06669354, -0.22753485,  0.24793473],
          [ 0.17

**LeNet-5 实战**

In [11]:
#keras高级API实现
network = Sequential([ # 网络容器
    layers.Conv2D(6,kernel_size=3,strides=1), # 第一个卷积层, 6个3x3卷积核
    layers.MaxPooling2D(pool_size=2,strides=2), # 高宽各减半的池化层
    layers.ReLU(), # 激活函数
    layers.Conv2D(16,kernel_size=3,strides=1), # 第二个卷积层, 16个3x3卷积核
    layers.MaxPooling2D(pool_size=2,strides=2), # 高宽各减半的池化层
    layers.ReLU(), # 激活函数
    layers.Flatten(), # 打平层，方便全连接层处理

    layers.Dense(120, activation='relu'), # 全连接层，120个节点
    layers.Dense(84, activation='relu'), # 全连接层，84节点
    layers.Dense(10) # 全连接层，10个节点
                    ])
# build一次网络模型，给输入X的形状，其中4为随意给的batchsz
network.build(input_shape=(4, 28, 28, 1))
# 统计网络信息
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (4, 26, 26, 6)            60        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (4, 13, 13, 6)            0         
_________________________________________________________________
re_lu (ReLU)                 (4, 13, 13, 6)            0         
_________________________________________________________________
conv2d_2 (Conv2D)            (4, 11, 11, 16)           880       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (4, 5, 5, 16)             0         
_________________________________________________________________
re_lu_1 (ReLU)               (4, 5, 5, 16)             0         
_________________________________________________________________
flatten_1 (Flatten)          (4, 400)                 

In [13]:
network.compile(optimizer='adam',
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],)
network.fit(x_train, y_train, epochs=5)
network.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5


ValueError: ignored

**手动实现**

In [14]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

network = Sequential([ # 网络容器
    layers.Conv2D(6,kernel_size=3,strides=1), # 第一个卷积层, 6个3x3卷积核
    layers.MaxPooling2D(pool_size=2,strides=2), # 高宽各减半的池化层
    layers.ReLU(), # 激活函数
    layers.Conv2D(16,kernel_size=3,strides=1), # 第二个卷积层, 16个3x3卷积核
    layers.MaxPooling2D(pool_size=2,strides=2), # 高宽各减半的池化层
    layers.ReLU(), # 激活函数
    layers.Flatten(), # 打平层，方便全连接层处理

    layers.Dense(120, activation='relu'), # 全连接层，120个节点
    layers.Dense(84, activation='relu'), # 全连接层，84节点
    layers.Dense(10) # 全连接层，10个节点
                    ])
# build一次网络模型，给输入X的形状，其中4为随意给的batchsz
network.build(input_shape=(4, 28, 28, 1))
# 统计网络信息
network.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (4, 26, 26, 6)            60        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (4, 13, 13, 6)            0         
_________________________________________________________________
re_lu_2 (ReLU)               (4, 13, 13, 6)            0         
_________________________________________________________________
conv2d_4 (Conv2D)            (4, 11, 11, 16)           880       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (4, 5, 5, 16)             0         
_________________________________________________________________
re_lu_3 (ReLU)               (4, 5, 5, 16)             0         
_________________________________________________________________
flatten_2 (Flatten)          (4, 400)                 

In [15]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = network(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(gradients, network.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = network(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        images=tf.expand_dims(images,axis=3)
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_images=tf.expand_dims(test_images,axis=3)
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Test Loss: {:.4f}, Test Accuracy: {:.4f}'
    print (template.format(epoch+1,
        train_loss.result(),
        train_accuracy.result()*100,
        test_loss.result(),
        test_accuracy.result()*100))

Epoch 1, Loss: 0.1936, Accuracy: 94.1083, Test Loss: 0.0818, Test Accuracy: 97.5700
Epoch 2, Loss: 0.1287, Accuracy: 96.0383, Test Loss: 0.0617, Test Accuracy: 98.1000
Epoch 3, Loss: 0.1008, Accuracy: 96.8889, Test Loss: 0.0615, Test Accuracy: 98.0733
Epoch 4, Loss: 0.0848, Accuracy: 97.3808, Test Loss: 0.0558, Test Accuracy: 98.2450
Epoch 5, Loss: 0.0736, Accuracy: 97.7127, Test Loss: 0.0537, Test Accuracy: 98.3100


**加入BN层**

In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

network = Sequential([ # 网络容器
    layers.Conv2D(6,kernel_size=3,strides=1),
    # 插入BN层
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2,strides=2),
    layers.ReLU(),
    layers.Conv2D(16,kernel_size=3,strides=1),
    # 插入BN层
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2,strides=2),
    layers.ReLU(),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    # 此处也可以插入BN层
    layers.Dense(84, activation='relu'), 
    # 此处也可以插入BN层
    layers.Dense(10)
])
# build一次网络模型，给输入X的形状，其中4为随意给的batchsz
network.build(input_shape=(4, 28, 28, 1))
# 统计网络信息
network.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (4, 26, 26, 6)            60        
_________________________________________________________________
batch_normalization (BatchNo (4, 26, 26, 6)            24        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (4, 13, 13, 6)            0         
_________________________________________________________________
re_lu_4 (ReLU)               (4, 13, 13, 6)            0         
_________________________________________________________________
conv2d_6 (Conv2D)            (4, 11, 11, 16)           880       
_________________________________________________________________
batch_normalization_1 (Batch (4, 11, 11, 16)           64        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (4, 5, 5, 16)            

In [17]:
model.compile(optimizer='adam',
      loss=losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'],)

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0393 - accuracy: 0.9886
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0304 - accuracy: 0.9907
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0279 - accuracy: 0.9909
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0251 - accuracy: 0.9924
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0212 - accuracy: 0.9932
313/313 - 1s - loss: 0.0940 - accuracy: 0.9812


[0.0940319374203682, 0.9811999797821045]

**cifar10+VGG**

In [18]:
import  tensorflow as tf
from    tensorflow.keras import layers, optimizers, datasets, Sequential
import  os

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
tf.random.set_seed(2345)

def preprocess(x, y):
    # [0~1]
    x = 2*tf.cast(x, dtype=tf.float32) / 255.-1
    y = tf.cast(y, dtype=tf.int32)
    return x,y

(x,y), (x_test, y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)
print(x.shape, y.shape, x_test.shape, y_test.shape)

170508288/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


In [19]:
train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.map(preprocess).batch(64)

sample = next(iter(train_db))
print('sample:', sample[0].shape, sample[1].shape,
      tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

network=Sequential([
     # unit 1
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 2
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 3
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 4
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 5
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    layers.Reshape([512]),
    
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(10, activation=None),
])

network.compile(optimizer='adam',
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],)

network.fit(train_db, epochs=50,validation_data=test_db,validation_freq=5)

sample: (128, 32, 32, 3) (128,) tf.Tensor(-1.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)
Epoch 1/50
391/391 [==============================] - 40s 92ms/step - loss: 2.3015 - accuracy: 0.1017
Epoch 2/50
391/391 [==============================] - 34s 88ms/step - loss: 2.2788 - accuracy: 0.1124
Epoch 3/50
391/391 [==============================] - 35s 88ms/step - loss: 1.6999 - accuracy: 0.3302
Epoch 4/50
391/391 [==============================] - 35s 88ms/step - loss: 1.3240 - accuracy: 0.5104
Epoch 5/50
391/391 [==============================] - 38s 98ms/step - loss: 1.0583 - accuracy: 0.6170 - val_loss: 0.9958 - val_accuracy: 0.6430
Epoch 6/50
391/391 [==============================] - 34s 88ms/step - loss: 0.8926 - accuracy: 0.6838
Epoch 7/50
391/391 [==============================] - 35s 88ms/step - loss: 0.7596 - accuracy: 0.7295
Epoch 8/50
391/391 [==============================] - 34s 87ms/step - loss: 0.6601 - accuracy: 0.7666
Epoch 9/50
391/391 [=========